In [196]:
from selenium import webdriver # type: ignore
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [268]:
options = Options()
options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

In [270]:
# Navigate to the URL
driver.get("https://bookings.malaysiaairlines.com/package/search-hotel?To=180008&DepartureDate=2024-10-01&ReturnDate=2024-10-03&Currency=MYR&CultureCode=en-US&RoomCount=1&PaxInfos%5B0%5D.AdultCount=1&PaxInfos%5B0%5D.ChildCount=1&PaxInfos%5B0%5D.ChildAges%5B0%5D=2&ProductId=a3fe407a-2347-40cd-90b7-7d18b1a1fdc3&StopoverDirection=0&RegionCode=Global&CountryCode=my-my&SkipPos=True")


In [272]:
soup =BeautifulSoup(driver.page_source, "html.parser")
print(soup)

<html class="" lang="en-US"><head class="at-element-marker"><style type="text/css">.swal-icon--error{border-color:#f27474;-webkit-animation:animateErrorIcon .5s;animation:animateErrorIcon .5s}.swal-icon--error__x-mark{position:relative;display:block;-webkit-animation:animateXMark .5s;animation:animateXMark .5s}.swal-icon--error__line{position:absolute;height:5px;width:47px;background-color:#f27474;display:block;top:37px;border-radius:2px}.swal-icon--error__line--left{-webkit-transform:rotate(45deg);transform:rotate(45deg);left:17px}.swal-icon--error__line--right{-webkit-transform:rotate(-45deg);transform:rotate(-45deg);right:16px}@-webkit-keyframes animateErrorIcon{0%{-webkit-transform:rotateX(100deg);transform:rotateX(100deg);opacity:0}to{-webkit-transform:rotateX(0deg);transform:rotateX(0deg);opacity:1}}@keyframes animateErrorIcon{0%{-webkit-transform:rotateX(100deg);transform:rotateX(100deg);opacity:0}to{-webkit-transform:rotateX(0deg);transform:rotateX(0deg);opacity:1}}@-webkit-key

In [274]:
listings =soup.find_all("div",class_="fh-tabs__overview")
len(listings)

74

In [276]:
card=listings[1]
#Rating
Ratings = []
for listing in listings:
    rating_div = listing.find("div", class_="rating")
    if rating_div:
        spans = rating_div.find_all("span")
        num_spans = len(spans)
        Ratings.append(num_spans)

Data_ratings=pd.DataFrame(Ratings,columns=["Ratings"])
Data_ratings=Data_ratings*2
#Price
Price=card.find("div", class_="price").text.replace("\n","")
print(Data_ratings)

    Ratings
0        10
1         8
2         6
3         6
4         6
..      ...
69        6
70        8
71        8
72        8
73        8

[74 rows x 1 columns]


In [278]:
#Place Name
Name=card.find("h2").text.replace("\n","")
print(Name)

Mercure Kuala Lumpur Shaw Parade Hotel  


In [280]:
#Date check in and check out
Checkbox=soup.find("div",class_="page-title-tabs")
len(Checkbox)

1

In [282]:
#Place
place=Checkbox.find("h1").text.replace("\n","")
Place=place.replace("Hotels in "," ")
print(Place)

 Kuala Lumpur


In [284]:
#Check_In and Checkout
url="https://bookings.malaysiaairlines.com/package/search-hotel?To=180008&DepartureDate=2024-10-01&ReturnDate=2024-10-03&Currency=MYR&CultureCode=en-US&RoomCount=1&PaxInfos%5B0%5D.AdultCount=1&PaxInfos%5B0%5D.ChildCount=1&PaxInfos%5B0%5D.ChildAges%5B0%5D=2&ProductId=a3fe407a-2347-40cd-90b7-7d18b1a1fdc3&StopoverDirection=0&RegionCode=Global&CountryCode=my-my&SkipPos=True"
date_regex = r"(\d{4}-\d{2}-\d{2})"
Check_In = re. search(r"DepartureDate=" + date_regex, url).group(1)
Check_Out = re. search(r"ReturnDate=" + date_regex, url).group(1)
print(Check_In)
print(Check_Out)

2024-10-01
2024-10-03


In [286]:
#Pax
pax = Checkbox.find_all("span", "icon-user")
pax=2
print(pax)

2


In [288]:
def get_hotel_listings(card):
    
    #Name
    Name=card.find("h2").text.replace("\n","") 
    #Price
    price_element = card.find("div", class_="price")
    Price = float(price_element.text.replace("MYR ", "").replace(",", ""))if price_element else 0
    Price = Price / 2

    hotel_listings = (Name,Price)
    return hotel_listings
    

In [290]:
hotels = []
listings =soup.find_all("div",class_="fh-tabs__overview")

for card in listings:
    hotel_lists = get_hotel_listings(card)
    if hotel_lists:
        hotels.append(hotel_lists)

In [292]:
print(hotels)

[('AVANI Sepang Goldcoast Resort  ', 491.09), ('Mercure Kuala Lumpur Shaw Parade Hotel  ', 308.79), ('AnCasa Hotel Kuala Lumpur  ', 230.335), ('Cititel Mid Valley  ', 276.915), ('Holiday Inn Express Kuala Lumpur City Centre  ', 307.74), ('The Gardens - A St Giles Signature Hotel & Residences, Kuala Lumpur  ', 511.125), ('Shangri-La Hotel Kuala Lumpur  ', 712.135), ('InterContinental Kuala Lumpur  ', 1209.935), ('Four Seasons Hotel Kuala Lumpur  ', 1452.29), ('Meliá Kuala Lumpur  ', 386.91), ('Sunway Velocity Hotel Kuala Lumpur  ', 410.405), ('Hotel Indigo Kuala Lumpur On The Park, an IHG Hotel  ', 583.275), ('Sunway Lagoon Hotel  ', 824.045), ('Sunway Putra Hotel, Kuala Lumpur  ', 387.725), ('Grand Millennium Kuala Lumpur  ', 659.17), ('Berjaya Times Square Hotel, Kuala Lumpur  ', 448.74), (' Ramada Suites Kuala Lumpur City Centre  ', 309.365), ('Tamu Hotel & Suites Kuala Lumpur  ', 318.33), ('Grand Ion Delemen Hotel  ', 214.145), ('Grand Bluewave Hotel Shah Alam  ', 331.17), ('Dorsett

In [294]:
Data=pd.DataFrame(hotels,columns=["Name Hotel","Price(Rm)"])
Data_1=Data_ratings.join(Data)
Data_1

,Ratings,Name Hotel,Price(Rm)
0,10,AVANI Sepang Goldcoast Resort,491.090
1,8,Mercure Kuala Lumpur Shaw Parade Hotel,308.790
2,6,AnCasa Hotel Kuala Lumpur,230.335
3,6,Cititel Mid Valley,276.915
4,6,Holiday Inn Express Kuala Lumpur City Centre,307.740
...,...,...,...
69,6,Park Inn by Radisson Putrajaya,250.950
70,8,Concorde Hotel Shah Alam,261.300
71,8,Fahrenheit Suites Kuala Lumpur,262.265
72,8,Mercure Living Putrajaya,265.775


In [296]:
Type="MHHolidays"
Duplicate_Data=[[pax,Check_In,Check_Out,Place,Type]]
Dummy_Data=pd.DataFrame(Duplicate_Data,columns=["Pax","Check In","Check Out","Place","Web"])
new_Duplicate_Data = Dummy_Data.reindex(index=Dummy_Data.index.repeat(len(listings))).reset_index(drop=True)
print(new_Duplicate_Data)

    Pax    Check In   Check Out          Place         Web
0     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
1     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
2     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
3     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
4     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
..  ...         ...         ...            ...         ...
69    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
70    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
71    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
72    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays
73    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays

[74 rows x 5 columns]


In [298]:
Final_Data =new_Duplicate_Data.join(Data_1)
print(Final_Data)

    Pax    Check In   Check Out          Place         Web  Ratings  \
0     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays       10   
1     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        8   
2     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        6   
3     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        6   
4     2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        6   
..  ...         ...         ...            ...         ...      ...   
69    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        6   
70    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        8   
71    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        8   
72    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        8   
73    2  2024-10-01  2024-10-03   Kuala Lumpur  MHHolidays        8   

                                        Name Hotel  Price(Rm)  
0                  AVANI Sepang Goldcoast Resort      491.090  
1         Mercure K

In [300]:
Final_Data.to_csv(f"D:/WebScraping/MHHolidays_{Check_In}_{Check_Out}_{Place}.csv",index=False,mode="w")